# BPR on movielens 100k

In [4]:
import numpy as np
import tensorflow as tf
import os
import random
from collections import defaultdict

## 载入数据集

In [5]:
import numpy
import tensorflow as tf
import os
import random
from collections import defaultdict

def load_data(data_path):
    '''
    Returns: 每个用户看过的电影
    '''
    
    user_ratings = defaultdict(set)
    max_u_id = -1
    max_i_id = -1 
    with open(data_path, 'r') as f:
        for line in f.readlines():
            u, i, _, _ = line.split("\t")
            u = int(u)
            i = int(i)
            user_ratings[u].add(i)
            max_u_id = max(u, max_u_id)
            max_i_id = max(i, max_i_id)
            
    print ("max_u_id:", max_u_id)
    print ("max_i_id:", max_i_id)
    
    return max_u_id, max_i_id, user_ratings
    
data_path = os.path.join(r'D:\CS\dataset\Recommend System\movielens-100k\ml-100k', 'u.data')
user_count, item_count, user_ratings = load_data(data_path)

max_u_id: 943
max_i_id: 1682


## 生成测试集

In [6]:
def generate_test(user_ratings):
    user_test = dict()
    for u, i_list in user_ratings.items():
        user_test[u] = random.sample(user_ratings[u], 1)[0]
    return user_test
    if asdad > asf:
user_ratings_test = generate_test(user_ratings)      # 测试集, 每个用户对应一个其评分过的电影

## 生成训练集
对于 <u,i,j>, 随机抽出用户u, 从user_ratings随机选取用户u看过的电影作为i, 再从所有电影中随机找出j, 同时保证j该用户u没有看过j. 即默认用户对于看过的电影的偏好大于没有看过的电影

In [7]:
def generate_train_batch(user_ratings, user_ratings_test, item_count, batch_size=512):
    '''
    Returns: ndarray: [ [u,i,j], [u,i,j] ...]
    '''
    t = []
    for b in range(batch_size):
        u = random.sample(user_ratings.keys(), 1)[0]
        i = random.sample(user_ratings[u], 1)[0]
        while i == user_ratings_test[u]:              # 避免选出的i已经存在测试集中
            i = random.sample(user_ratings[u], 1)[0]
        
        j = random.randint(1, item_count)             # 随机抽取一个 itemID
        while j in user_ratings[u]:                   # 应该保证j没有被当前用户评分过
            j = random.randint(1, item_count)
        t.append([u, i, j])
        
    return np.asarray(t)

In [8]:
def generate_test_batch(user_ratings, user_ratings_test, item_count):
    for u in user_ratings.keys():
        t = []
        i = user_ratings_test[u]
        for j in range(1, item_count+1):
            if not (j in user_ratings[u]):
                t.append([u, i, j])
        yield numpy.asarray(t)

## TF modeling

In [9]:
a = tf.Variable(tf.random_normal([4,3], stddev=0.1), name='a')
a = tf.expand_dims(a, -1)

In [10]:
a

<tf.Tensor 'ExpandDims:0' shape=(4, 3, 1) dtype=float32>

In [11]:
def bpr_mf(user_count, item_count, hidden_dim):
    u = tf.placeholder(tf.int32, [None])    # 接收训练样本中的 uID
    i = tf.placeholder(tf.int32, [None])    # 接收训练样本中的 iID
    j = tf.placeholder(tf.int32, [None])

    with tf.device("/cpu:0"):
        user_emb_w = tf.get_variable("user_emb_w", [user_count+1, hidden_dim], 
                            initializer=tf.random_normal_initializer(0, 0.1))
        item_emb_w = tf.get_variable("item_emb_w", [item_count+1, hidden_dim], 
                                initializer=tf.random_normal_initializer(0, 0.1))
        
        u_emb = tf.nn.embedding_lookup(user_emb_w, u)    # W中对应训练样本uID的行
        i_emb = tf.nn.embedding_lookup(item_emb_w, i)
        j_emb = tf.nn.embedding_lookup(item_emb_w, j)
    
    # MF predict: u_i > u_j
    x = tf.reduce_sum(tf.multiply(u_emb, (i_emb - j_emb)), 1, keep_dims=True)
    
    # AUC for one user:
    # reasonable iff all (u,i,j) pairs are from the same user

    # average AUC = mean( auc for each user in test set)
    mf_auc = tf.reduce_mean(tf.to_float(x > 0))
    
    # 正则化项, 平方和
    l2_norm = tf.add_n([           
            tf.reduce_sum(tf.multiply(u_emb, u_emb)), 
            tf.reduce_sum(tf.multiply(i_emb, i_emb)),
            tf.reduce_sum(tf.multiply(j_emb, j_emb))
        ])
    
    regulation_rate = 0.0001
    bprloss = - regulation_rate * l2_norm - tf.reduce_mean(tf.log(tf.sigmoid(x)))
    train_op = tf.train.GradientDescentOptimizer(0.01).minimize(bprloss)
    
    return u, i, j, mf_auc, bprloss, train_op

In [12]:
with tf.Graph().as_default(), tf.Session() as session:
    u, i, j, mf_auc, bprloss, train_op = bpr_mf(user_count, item_count, 20)
    session.run(tf.global_variables_initializer())
    for epoch in range(1, 4):
        _batch_bprloss = 0
        for k in range(1, 5000): # uniform samples from training set
            uij = generate_train_batch(user_ratings, user_ratings_test, item_count)

            _bprloss, _train_op = session.run([bprloss, train_op], 
                                feed_dict={u:uij[:,0], i:uij[:,1], j:uij[:,2]})
            _batch_bprloss += _bprloss
        
        print ("epoch: ", epoch)
        print ("bpr_loss: ", _batch_bprloss / k)
        print ("_train_op")

        user_count = 0
        _auc_sum = 0.0

        # each batch will return only one user's auc
        for t_uij in generate_test_batch(user_ratings, user_ratings_test, item_count):
            _auc, _test_bprloss = session.run([mf_auc, bprloss],
                                               feed_dict={u:t_uij[:,0], i:t_uij[:,1], j:t_uij[:,2]})
            user_count += 1
            _auc_sum += _auc
        print ("test_loss: ", _test_bprloss, "test_auc: ", _auc_sum/user_count)
        print ("________________________________________________________________________________")
        
    variable_names = [v.name for v in tf.trainable_variables()]
    values = session.run(variable_names)
    for k,v in zip(variable_names, values):
        print("Variable: ", k)
        print("Shape: ", v.shape)
        print(v)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
epoch:  1
bpr_loss:  0.662683289905719
_train_op
test_loss:  0.6119885 test_auc:  0.5035516543146954
________________________________________________________________________________
epoch:  2
bpr_loss:  0.6620350534092262
_train_op
test_loss:  0.61047125 test_auc:  0.5036232185136222
________________________________________________________________________________
epoch:  3
bpr_loss:  0.6613667560687277
_train_op
test_loss:  0.60887396 test_auc:  0.5036008184191041
________________________________________________________________________________
Variable:  user_emb_w:0
Shape:  (944, 20)
[[ 0.06405131 -0.10947271 -0.02696753 ... -0.12656157  0.03924455
   0.11468504]
 [-0.1004385  -0.04399877  0.01715755 ...  0.00125282  0.00166603
  -0.12702945]
 [ 0.1901028   0.03350162  0.09161587 ...  0.06237383  0.13398826
   0.22615623]
 ...
 [-0.07719813  0.1037305   0.09040267 ...  0.03548682  0.14065503
   0.16374888]
 [-0.1

## Prediction

In [21]:
session1 = tf.Session()
u1_dim = tf.expand_dims(values[0][0], 0)    # [20] reshape成2阶ndarray[1, 20], 以在下一步进行矩阵乘法
u1_all = tf.matmul(u1_dim, values[1], transpose_b=True)
result_1 = session1.run(u1_all)
print(result_1)       # W中第一个用户的电影BPR排序结果

[[ 0.0302686  -0.03186373 -0.04049495 ... -0.03867592 -0.03704941
   0.04978933]]


In [23]:
print("以下是给用户0的推荐：")
p = numpy.squeeze(result_1)
p[numpy.argsort(p)[:-5]] = 0
for index in range(len(p)):
    if p[index] != 0:
        print (index, p[index])

以下是给用户0的推荐：
470 0.10199209
739 0.094658285
1082 0.10206555
1098 0.093990594
1439 0.089905776
